# 银行营销预测

学习的思路

从对数据集进行探索，数据预处理和特征工程，到学习模型的评估与选择

较为完整的展示了解决分类问题

####  赛题意义 [第一步]

#### 数据说明及评价函数 [第二步]

## 导入相应的库

基本的数据处理库

In [1]:
import pandas as pd
import numpy as py

sklearn的相关模块

In [2]:
from sklearn import model_selection

In [3]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# 认识数据

数据集的输入变量是20个特征量，分为数值变量（numeric）和分类（categorical）变量。 

输出变量为 0 1 ，即是否接受

## 数据集的获取

读取数据时，解码使用GB2312是中国规定的汉字编码，也可以说是简体中文的字符集编码gbk，utf-8

载入数据

In [5]:
train_data = pd.read_csv("dataset/trainx.csv")
train_datay = pd.read_csv("dataset/trainy.csv")
test_data = pd.read_csv("dataset/testx.csv")

## 数据初步分析

行列初窥,之后也会有相关数据的分析，更多的就是训练集的分析

In [6]:
train_data.shape

(8000, 20)

In [7]:
test_data.shape

(2000, 20)

In [8]:
train_data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype='object')

In [9]:
test_data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype='object')

使用info()函数和describe()函数查看数据集的基本信息

查看数据类型  train_data.info()与train_data.dtypes有点类似，我比较习惯使用train_data.dtypes

In [10]:
#train_data.info()

In [11]:
train_data.dtypes

age               float64
job               float64
marital           float64
education         float64
default           float64
housing           float64
loan              float64
contact           float64
month             float64
day_of_week       float64
duration          float64
campaign          float64
pdays             float64
previous          float64
poutcome          float64
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
dtype: object

In [12]:
train_data.describe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,8000.000000,7931.000000,7983.000000,7629.000000,6713.0,7799.000000,7799.000000,8000.000000,8000.000000,8000.000000,8000.0000,8000.000000,8000.000000,8000.000000,8000.00000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,40.328625,3.756021,1.207566,3.715559,0.0,0.537120,0.157071,0.285625,5.689875,1.996125,377.3635,2.371000,892.498750,0.300625,0.98525,-0.464038,93.478831,-40.201275,3.005255,5137.635338
std,11.752580,3.566145,0.611329,2.070302,0.0,0.498652,0.363891,0.451740,2.206027,1.393145,353.8095,2.470197,307.292248,0.675505,0.46077,1.717363,0.626933,5.295826,1.884575,85.957972
min,17.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,2.000000,0.000000,0.0000,1.000000,0.000000,0.000000,0.00000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,0.000000,1.000000,2.000000,0.0,0.000000,0.000000,0.000000,4.000000,1.000000,140.7500,1.000000,999.000000,0.000000,1.00000,-1.800000,92.893000,-42.700000,1.250000,5076.200000
50%,38.000000,3.000000,1.000000,3.000000,0.0,1.000000,0.000000,0.000000,5.000000,2.000000,256.0000,2.000000,999.000000,0.000000,1.00000,-0.100000,93.444000,-41.800000,4.076000,5191.000000
75%,48.000000,7.000000,2.000000,6.000000,0.0,1.000000,0.000000,1.000000,7.000000,3.000000,507.2500,3.000000,999.000000,0.000000,1.00000,1.400000,93.994000,-36.400000,4.959000,5228.100000
max,98.000000,10.000000,2.000000,6.000000,0.0,1.000000,1.000000,1.000000,11.000000,4.000000,4199.0000,43.000000,999.000000,6.000000,2.00000,1.400000,94.767000,-26.900000,5.045000,5228.100000


## 数据集预处理

数据清洗

缺失值分析

In [13]:
train_data.isnull().sum()

age                  0
job                 69
marital             17
education          371
default           1287
housing            201
loan               201
contact              0
month                0
day_of_week          0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
dtype: int64

In [14]:
#直接删除空值
#train_data = train_data.dropna()

In [15]:
train_data=train_data.fillna(method="ffill")

In [16]:
test_data=test_data.fillna(method="ffill")

## 数据可视化

In [17]:
#train_data.plot(kind='box',subplots=True,layout=(30,30),sharex=False,sharey=False)

In [18]:
#train_data.hist()

## 特征工程

In [19]:
array=train_data.values
arrayy=train_datay.values

In [20]:
X=array[:,0:20]
Y=arrayy[:,0]
test_size=0.20
seed=7
X_train,X_test,Y_train,Y_test=model_selection.train_test_split(X,Y,test_size=test_size,random_state=seed)

In [21]:
X_train.shape

(6400, 20)

In [22]:
X_test.shape

(1600, 20)

In [23]:
Y_train.shape

(6400,)

In [24]:
Y_test.shape

(1600,)

## 模型的训练

In [25]:
scoring="accuracy"

In [26]:
models=[]

In [27]:
models.append(("逻辑回归",LogisticRegression()))
models.append(("线性辨别分析",LinearDiscriminantAnalysis()))
models.append(("K近邻",KNeighborsClassifier()))
models.append(("分类回归树",DecisionTreeClassifier()))
models.append(("朴素贝叶斯的高斯模型",GaussianNB()))
models.append(("支持向量机",SVC()))

In [28]:
results=[]

In [29]:
names=[]

In [30]:
for name,model in models:
    kflod=model_selection.KFold(n_splits=10,random_state=seed)
    cv_results=model_selection.cross_val_score(model,X_train,Y_train,cv=kflod,scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg="%s:%f(%f)"%(name,cv_results.mean(),cv_results.std())
    print(msg)

逻辑回归:0.846250(0.016044)
线性辨别分析:0.832656(0.017236)


D:\Users\lsk\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
D:\Users\lsk\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
D:\Users\lsk\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
D:\Users\lsk\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
D:\Users\lsk\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
D:\Users\lsk\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
D:\Users\lsk\Anaconda3\lib\site-pa

K近邻:0.853125(0.018355)
分类回归树:0.826562(0.016223)
朴素贝叶斯的高斯模型:0.778594(0.021797)
支持向量机:0.805469(0.023210)


### 最优模型选取

In [31]:
good_mod=KNeighborsClassifier()
good_mod.fit(X_train,Y_train)
pre = good_mod.predict(X_test)

In [32]:
accuracy_score(Y_test,pre)

0.846875

In [33]:
confusion_matrix(Y_test,pre)

array([[668, 137],
       [108, 687]], dtype=int64)

## 模型的预测输出

In [34]:
test_data.shape

(2000, 20)

In [35]:
pre=good_mod.predict(test_data)

In [36]:
df= pd.DataFrame(pre)

In [37]:
df.to_csv("dataset/testy.csv",index=False)

因为数据集的预处理没有做好，只是简单的复制前一个数字

造成数据集的最后得分不是很高，只是80%左右